In [6]:
# Libraries
import pandas as pd
import random
import json
from datasets import load_dataset
import re
import math
import random
import spacy

## Getting instructions

In [7]:
dataset = load_dataset("gsm8k", "main")
dataset["test"][5]

Found cached dataset gsm8k (/Users/iv/.cache/huggingface/datasets/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba)


  0%|          | 0/2 [00:00<?, ?it/s]

{'question': 'Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?',
 'answer': 'The discount price of one glass is 60/100 * 5 = $<<60/100*5=3>>3.\nIf every second glass is cheaper, that means Kylar is going to buy 16 / 2 = <<16/2=8>>8 cheaper glasses.\nSo for the cheaper glasses, Kylar is going to pay 8 * 3 = $<<8*3=24>>24.\nAnd for the regular-priced glasses, Kylar will pay 8 * 5 = $<<8*5=40>>40.\nSo in total Kylar needs to pay 24 + 40 = $<<24+40=64>>64 for the glasses he wants to buy.\n#### 64'}

In [8]:
def get_random_test_object(dataset):
    if "test" in dataset and len(dataset["test"]) > 0:
        random_index = random.randint(0, len(dataset["test"]) - 1)
        test_object = dataset["test"][random_index]
        
        # Assuming each object in the dataset has 'question' and 'answer' keys
        question = test_object.get("question", "No question found")
        answer = test_object.get("answer", "No answer found")
        
        return question, answer
    else:
        return None, None

In [9]:
def get_random_mutation(csv_file_path):
    try:
        df = pd.read_csv(csv_file_path, header=None, encoding='utf-8', delimiter='.') 
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file_path, header=None, encoding='ISO-8859-1', delimiter='.') 

    random_prompt = random.choice(df[1].tolist())
    return random_prompt


In [10]:
def get_random_mutation_txt(txt_file_path):
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove any leading/trailing whitespace and filter out empty lines
    prompts = [line.strip() for line in lines if line.strip()]
    
    if prompts:
        return random.choice(prompts)
    else:
        return "No mutation prompts found."

In [11]:
# write the task description here:
task_description = "Generate an instruction on how to solve the problem, based on the given question "

In [12]:
task_description1 = "Generate an instruction, or advice on how to solve a problem"

#### Hide

In [11]:
# getting mutated prompts
question, answer = get_random_test_object(dataset)

In [12]:
print(question)

Jackie is trying to decide whether to do her taxes herself or hire an accountant. If she does the taxes herself, she'll be able to do 3 fewer hours of freelance work, losing $35/hour in missed income. The accountant charges $90. How much more money will she have if she hires the accountant?


In [13]:
print(answer)

First find the total lost revenue if Jackie does her taxes herself: $35/hour * 3 hours = $<<35*3=105>>105
Then subtract the accountant's charge to find how much money Janet saves: $105 - $90 = $<<105-90=15>>15
#### 15


## Instruction Generation

In [61]:
def check_answer(answer, output):
    if not (isinstance(answer, str) and isinstance(output, str)):
        raise TypeError("Must be of type str")
    if re.search("\s" + answer +"\s*", output):
        # print("answer is correct")
        return 1
    # print("answer is wrong")
    return 0

In [62]:
def get_logic(answer):
    return re.findall("<<(.*?)>>", answer)

In [63]:
def tokenise_logic(logic):
    tokenised_logic = []
    operators = "+-/*="
    current_token = ""
    
    for c in logic:
        if c in operators:
            tokenised_logic.append(current_token)
            tokenised_logic.append(c)
            current_token = ""
        elif c == '.' or c.isnumeric():
            current_token += c
    tokenised_logic.append(current_token)
    
    return tokenised_logic

In [64]:
def check_logic_sentence(line, logic_tokens):
    i = 0

    for token in logic_tokens:
        if token in line:
            i += 1

    if len(logic_tokens) == i:
        return True
    return False

In [134]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

class llamathwiz:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
        self.model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
        self.gen = 0
        
    def generate_intructions(self, question, task_description, max_tokens=100):
        formatted_input = f"{task_description} {question}"
        input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids
        outputs = model.generate(input_ids, max_new_tokens = max_tokens)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return generated_text
    
    def generate_instructions1(self, num_instructions, max_tokens=1000, temperature=1.5):
        instructions = []
        for num in range(num_instructions):
            formatted_input = "Generate instructions on how to solve a math problem"
            input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids
            outputs = model.generate(input_ids, 
                                     max_new_tokens=int(max_tokens),
                                     temperature=temperature)
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            instructions.append(generated_text)

        return instructions

    def process_with_llm(self, prompt):
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        output = model.generate(input_ids, max_length=1000)
        response = tokenizer.decode(output[0], skip_special_tokens=True)

        return response
    
    def apply_mutation(self, instruction, mutation_prompt):
        # Example mutation - this can be customized based on your mutation logic
        return f"{mutation_prompt} {instruction}"
    
    def fitness(self, database_answer, output):
        final_answer = database_answer.split()[-1]

        score = 0

        logic = get_logic(database_answer)

        tokenised_logic_sentences = []

        for l in logic:
            tokenised_logic_sentences.append(tokenise_logic(l))

        total_score = 1 + len(tokenised_logic_sentences)

        for line in output.split("."):
            for sentence in tokenised_logic_sentences:
                if check_logic_sentence(line, sentence):
                    tokenised_logic_sentences.pop(tokenised_logic_sentences.index(sentence))
                    score += 1
                    break
        last_line = line

        num_answer = 'a'

        for word in last_line.split(" "):
            try:
                num_answer = str(int(word))
            except:
                pass

        score += check_answer(final_answer, num_answer)
        return score/total_score
    
    def iteration_first(self, num_instructions=10):
        self.gen = 1
        self.generated_prompts = []
        self.scores = []
        
        for _ in range(num_instructions):
            question, database_answer = get_random_test_object(dataset)  # Fetch a random question from your dataset
            instruction = self.generate_intructions(question, task_description)
            mutation_prompt = get_random_mutation_txt("./Prompt-Engineering-OpenDI/mutation_prompts.txt")
            mutated_instruction = self.apply_mutation(instruction, mutation_prompt)
            self.generated_prompts.append(mutated_instruction)
            processed_output = self.process_with_llm(mutated_instruction)
            self.scores.append(self.fitness(database_answer, processed_output))
            
    
    def iteration_execute(self):
        self.gen += 1
        self.generated_prompts = []
        self.scores = []

        for instruction in self.best_instructions:
            question, database_answer = get_random_test_object(dataset)  # Fetch a random question from your dataset
            mutation_prompt = get_random_mutation_txt("./Prompt-Engineering-OpenDI/mutation_prompts.txt")
            mutated_instruction = self.apply_mutation(instruction, mutation_prompt)
            self.generated_prompts.append(mutated_instruction)
            processed_output = self.process_with_llm(mutated_instruction)
            self.scores.append(self.fitness(database_answer, processed_output))
    
    def iteration_prepare(self):
        self.best_instructions = []
        self.find_best_scores(self.scores)
        self.best_instructions.append(self.replace_pos(self.best_instructions[0], self.best_instructions[1], ['ADV', 'ADJ', 'NOUN']))
        self.best_instructions.append(self.replace_pos(self.best_instructions[1], self.best_instructions[0], ['ADV', 'ADJ', 'NOUN']))
        for instr in self.generate_instructions1(4):
            self.best_instructions.append(instr)
        for i in range(2):
            mutation_prompt = get_random_mutation_txt("./Prompt-Engineering-OpenDI/mutation_prompts.txt")
            self.best_instructions.append(self.apply_mutation(self.best_instructions[i], mutation_prompt))
        
    def find_best_scores(self, scores):
        for i in range(2):
            maximum_val = 0
            maximum_index = 0
            while i < len(scores):
                if scores[i] > maximum_val:
                    maximum_val = scores[i]
                    maximum_index = i
                i += 1
            scores.pop(maximum_index)
            print(self.generated_prompts[maximum_index])
            self.best_instructions.append(self.generated_prompts.pop(maximum_index))
            
    def replace_pos(self, string1, string2, POS):

        """
        Replace tokens of specific parts of speech (POS) in string1 with corresponding
        tokens of the same POS from string2.

        Parameters:
        - string1 (str): The input string where certain POS will be replaced.
        - string2 (str): The reference string from which POS replacements will be taken.
        - POS (list): A list of POS tags to identify which tokens to replace in string1.

        Returns:
        - str: The modified string with replacements of tokens based on specified POS tags.

        Possible POS:
        "ADJ": "adjective",
        "ADP": "adposition",
        "ADV": "adverb",
        "AUX": "auxiliary",
        "CONJ": "conjunction",
        "CCONJ": "coordinating conjunction",
        "DET": "determiner",
        "INTJ": "interjection",
        "NOUN": "noun",
        "NUM": "numeral",
        "PART": "particle",
        "PRON": "pronoun",
        "PROPN": "proper noun",
        "PUNCT": "punctuation",
        "SCONJ": "subordinating conjunction",
        "SYM": "symbol",
        "VERB": "verb".
        """
        nlp = spacy.load("en_core_web_sm")
        doc1 = nlp(string1)
        doc2 = nlp(string2)

        new_tokens = []

        for token1 in doc1:
            if token1.pos_ in POS:
                #generating a list of all matching tokens
                matching_tokens = [token2.text for token2 in doc2 if token2.pos_ == token1.pos_]
                #use a random token if possible, or else use the same
                if matching_tokens:
                    new_token = random.choice(matching_tokens)
                else:
                    new_token = token1.text
            else:
                new_token = token1.text
            new_tokens.append(new_token)

        # Join the modified tokens to form the final string
        result = ' '.join(new_tokens)
        return result
    
    

In [135]:
ai = llamathwiz()

In [154]:
ai.iteration_first()

In [155]:
ai.scores

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.25, 0.0]

In [156]:
ai.iteration_prepare()

The above working out has some errors, here is a version with the errors fixed. Jenny buys 36 cookies / bag = 6 cookies a week. She puts 4 cookies in her son's lunch box 5 days a week so she puts 5 * 6 = 24 cookies in her son's lunch box. Her husband eats 1 cookie a day for 7 days so he eats 1 * 7 = 2 cookies a day. Jenny eats 6 cookies - 24 cookies = 6 cookies a week.
Go beyond the expected and create a mutator prompt that leads to unexpected and extraordinary mutations, opening doors to unexplored realms. Increase Specificity: If the original prompt is too general, like ’Tell me about X,’ the modified version could be, ’Discuss the history, impact, and current status of X.’ Billy sold 3 * 1 = 3 DVDs to his first customers. His next 2 customers bought 2 * 2 = 4 DVDs. His last 3 customers didn't buy any DVDs so he sold 3 + 4 + 4 = 9 DVDs.


In [157]:
len(ai.best_instructions)

10

In [158]:
ai.iteration_execute()

In [159]:
ai.scores

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]